In [1]:
import pickle
import numpy as np
import streamlit as st
from sklearn.ensemble import VotingClassifier
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [29]:
tf = pd.read_csv('Overview.csv')

tf['DeviceTimeStamp'] = pd.to_datetime(tf['DeviceTimeStamp'], format='%Y-%m-%dT%H:%M')

cv = pd.read_csv('CurrentVoltage.csv')

# convert to datetime datatype
cv['DeviceTimeStamp'] = pd.to_datetime(cv['DeviceTimeStamp'], format='%Y-%m-%dT%H:%M')

# merge the datasets

cv['DeviceTimeStamp'] = pd.to_datetime(cv['DeviceTimeStamp'],format = '%Y-%m-%d %H:%M:%S')
transformer = pd.merge(tf, cv, on='DeviceTimeStamp')

X = transformer.drop(['DeviceTimeStamp','MOG_A'],axis=1)
y = transformer['MOG_A']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.20,random_state=21)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Load your models
pickle_in2 = open("svc_model.pkl", "rb")
svc_classifier = pickle.load(pickle_in2)

pickle_in3 = open("knn_model.pkl", "rb")
knn_classifier = pickle.load(pickle_in3)

pickle_in6 = open("extra_trees_model.pkl", "rb")
et = pickle.load(pickle_in6)

pickle_in7 = open("adaboost_model.pkl", "rb")
ada = pickle.load(pickle_in7)

pickle_in8 = open("xgboost_model.pkl", "rb")
xgb = pickle.load(pickle_in8)



ensemble_model = VotingClassifier(estimators=[('dt', svc_classifier), ('lr', knn_classifier), ('knn', et), ('svm', ada), ('rf', xgb)], voting='hard')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions
predictions = ensemble_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

# prob_predictions = ensemble_model.predict_proba(X_test)

# # Get the probabilities for the positive class (class 1)
# positive_class_probs = prob_predictions[:, 1]

C:\Users\devaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AttributeError: predict_proba is not available when probability=False

In [3]:
print("Ensemble Model Accuracy:", accuracy)

Ensemble Model Accuracy: 0.9880283410701197


In [4]:
import pickle

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Save the ensemble model to a file using pickle
with open('ensemble_model.pkl', 'wb') as f:
    pickle.dump(ensemble_model, f)

print("Ensemble model saved to ensemble_model.pkl")

C:\Users\devaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Ensemble model saved to ensemble_model.pkl


In [6]:
# # Initialize an empty dictionary to store parameters for faulty predictions
# faulty_parameters = {}

# # Iterate over the dataset
# for index, row in transformer.iterrows():
#     # Transform the row similar to how you did for training
#     transformed_row = scaler.transform(row.drop(['DeviceTimeStamp','MOG_A']).values.reshape(1, -1))
    
#     # Make predictions using the ensemble model
#     prediction = ensemble_model.predict(transformed_row)[0]
    
#     # If the prediction is faulty (1), store the parameters
#     if prediction == 1:
#         # Convert the row to a tuple to use it as a dictionary key
#         parameters_tuple = tuple(row.drop(['DeviceTimeStamp','MOG_A']).values)
        
#         # If the parameters tuple is not already in the dictionary, initialize it as an empty list
#         if parameters_tuple not in faulty_parameters:
#             faulty_parameters[parameters_tuple] = []
        
#         # Append the index to the list of faulty predictions for these parameters
#         faulty_parameters[parameters_tuple].append(index)
        
#         # Check if the dictionary has reached the limit of 50 rows
#         if len(faulty_parameters) >= 50:
#             break

# # Now you have a dictionary where keys are parameter tuples and values are lists of indices with faulty predictions, limited to 50 rows


In [8]:
# print(faulty_parameters)

{}


In [9]:
# column_names = ['OTI', 'WTI', 'ATI', 'OLI', 'OTI_A', 'OTI_T', 'VL1', 'VL2', 'VL3', 'IL1', 'IL2', 'IL3', 'VL12', 'VL23', 'VL31', 'INUT']

# # Create a DataFrame
# df = pd.DataFrame(columns=column_names)

# # Add data to the DataFrame
# for key, value in faulty_parameters.items():
#     df.loc[len(df)] = key

# # Print the DataFrame
# print(df)

In [ ]:
# output = ensemble_model.predict(X_train)

In [ ]:
# print(output)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [35]:
filtered_df = transformer[transformer['MOG_A'] ==  0.0]

# Print the filtered DataFrame
print(filtered_df)



          DeviceTimeStamp   OTI  WTI   ATI    OLI  OTI_A  OTI_T  MOG_A    VL1  \
0     2019-06-25 13:06:00   0.0  0.0   0.0   37.0    0.0    0.0    0.0    0.0   
1     2019-06-27 10:49:00   0.0  0.0   0.0   38.0    0.0    0.0    0.0  238.7   
2     2019-06-27 10:51:00   0.0  0.0   0.0   38.0    0.0    0.0    0.0  238.4   
3     2019-06-27 10:52:00   0.0  0.0   0.0   39.0    0.0    0.0    0.0  239.9   
4     2019-06-27 10:52:00   0.0  0.0   0.0   39.0    0.0    0.0    0.0  239.9   
...                   ...   ...  ...   ...    ...    ...    ...    ...    ...   
20460 2020-04-13 23:30:00  31.0  1.0  28.0  100.0    0.0    0.0    0.0  246.4   
20461 2020-04-13 23:45:00  31.0  1.0  28.0  100.0    0.0    0.0    0.0  246.8   
20462 2020-04-14 00:00:00  31.0  1.0  28.0  100.0    0.0    0.0    0.0  247.8   
20463 2020-04-14 00:15:00  30.0  1.0  28.0  100.0    0.0    0.0    0.0  247.7   
20464 2020-04-14 00:30:00  30.0  1.0  28.0  100.0    0.0    0.0    0.0  248.4   

         VL2    VL3   IL1  

In [36]:
filtered_df = filtered_df.drop(['DeviceTimeStamp','MOG_A'],axis=1)

In [23]:
# filtered_df = filtered_df.astype(float)
output = ensemble_model.predict(filtered_df)
# filtered_df.dtypes

In [24]:
if 1.0 in output:
    print("There are 1 values in the output array.")
else:
    print("There are no 1 values in the output array.")

There are 1 values in the output array.


In [25]:
print(output)

[0. 0. 1. ... 1. 1. 1.]


In [32]:

from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pickle

# Define a custom colormap with varied colors
cmap = ListedColormap(['#0071C5', '#4EA2F2', '#A4C8FD', '#CDE3FD'])

# Train the logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict probabilities instead of classes
y_pred_proba_lr = logreg.predict_proba(X_test)

# Extract the probabilities for class 1 (assuming class 1 is the positive class)
y_pred_proba_positive = y_pred_proba_lr[:, 1]

# Save the model
with open('logistic_regression_model_prob.pkl', 'wb') as file:
    pickle.dump(logreg, file)


# print(y_pred_proba_positive)



y_pred_proba_positive_percent = [round(prob * 100, 2) for prob in y_pred_proba_positive]

# Print the rounded probabilities
print(y_pred_proba_positive_percent)


[2.88, 0.0, 90.51, 0.0, 0.0, 0.0, 6.69, 0.0, 0.0, 0.0, 78.43, 35.65, 0.0, 0.0, 0.0, 0.0, 39.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 97.08, 0.0, 0.0, 0.0, 4.54, 0.0, 0.0, 0.0, 0.0, 0.33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 17.97, 1.24, 1.8, 0.0, 11.28, 9.59, 0.0, 0.0, 7.73, 93.12, 0.0, 0.0, 0.0, 1.46, 0.0, 78.14, 98.78, 0.08, 0.0, 3.45, 0.0, 0.0, 0.0, 0.0, 0.0, 41.42, 0.0, 40.48, 19.06, 0.0, 76.89, 0.0, 0.0, 0.0, 0.0, 79.48, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.73, 0.0, 99.76, 0.0, 0.0, 0.0, 0.0, 0.0, 12.67, 0.0, 0.0, 0.0, 52.3, 2.36, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 1.8, 0.0, 0.0, 0.0, 69.31, 0.0, 0.0, 0.82, 14.47, 8.49, 0.0, 0.0, 0.0, 0.0, 72.24, 0.0, 0.0, 0.0, 0.0, 0.0, 4.67, 42.25, 0.0, 0.0, 0.0, 82.55, 0.0, 0.0, 84.98, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 38.96, 90.62, 19.87, 0.0, 0.0, 0.0, 79.78, 0.0, 0.0, 0.0, 0.0, 0.0, 14.72, 2.75, 0.0, 0.0, 24.67, 0.0, 18.36, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 75.22, 0.02, 0.0, 0.0, 0.0

C:\Users\devaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
y_pred_proba_lr = logreg.predict_proba(filtered_df)

# Extract the probabilities for class 1 (assuming class 1 is the positive class)
y_pred_proba_positive = y_pred_proba_lr[:, 1]
y_pred_proba_positive_percent = [round(prob * 100, 2) for prob in y_pred_proba_positive]

# Print the rounded probabilities
print(y_pred_proba_positive_percent)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [41]:

pickle_prob_1 = open("logistic_regression_model_prob.pkl", "rb")
lr_prob = pickle.load(pickle_prob_1)

# pickle_in3 = open("knn_model.pkl", "rb")
# knn_classifier = pickle.load(pickle_in3)

# pickle_in6 = open("extra_trees_model.pkl", "rb")
# et = pickle.load(pickle_in6)

# pickle_in7 = open("adaboost_model.pkl", "rb")
# ada = pickle.load(pickle_in7)

# pickle_in8 = open("xgboost_model.pkl", "rb")
# xgb = pickle.load(pickle_in8)



ensemble_model = VotingClassifier(estimators=[
    ('lr_prob', lr_prob)], voting='soft')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions
# predictions = ensemble_model.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, predictions)

prob_predictions = ensemble_model.predict_proba(filtered_df)

# Get the probabilities for the positive class (class 1)
positive_class_probs = prob_predictions[:, 1]

y_pred_proba_positive_percent = [round(prob * 100, 2) for prob in positive_class_probs]

# Print the rounded probabilities
print(y_pred_proba_positive_percent)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

C:\Users\devaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pickle

# Define a custom colormap with varied colors
cmap = ListedColormap(['#FFA500', '#FFD700', '#FFA07A', '#FF4500'])

# Create and fit the SVM model
svc = SVC(probability=True)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict_proba(X_test)

svc_train = round(svc.score(X_train, y_train) * 100, 2)
svc_accuracy = round(accuracy_score(y_pred_svc, y_test) * 100, 2)



# Save the model
with open('svc_model_prob.pkl', 'wb') as file:
    pickle.dump(svc, file)


# y_pred_proba_lr = logreg.predict_proba(X_test)

# Extract the probabilities for class 1 (assuming class 1 is the positive class)
y_pred_proba_positive = y_pred_svc[:, 1]

# print(y_pred_proba_positive)



y_pred_svc_positive_percent = [round(prob * 100, 2) for prob in y_pred_proba_positive]

# Print the rounded probabilities
print(y_pred_proba_positive_percent)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets